In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain mcp

In [0]:
dbutils.library.restartPython()

### MCP Server

In [0]:
server_code = r'''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("dbx-mcp-smoke-test")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Return a + b."""
    return a + b

if __name__ == "__main__":
    # Runs an MCP server over stdio (so a client can connect via pipes)
    mcp.run()
'''

path = "/tmp/mcp_server.py"
with open(path, "w") as f:
    f.write(server_code)

print("Wrote:", path)

In [0]:
import os, sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command=sys.executable,
    args=["/tmp/mcp_server.py"],
    env=os.environ.copy(),
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await session.list_tools()
        print("Tools:", [t.name for t in tools.tools])

        res = await session.call_tool("add", {"a": 2, "b": 3})
        print("add(2, 3) =>", res.content)


### FastMCP

In [0]:
server_code = r'''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("hello-server")

@mcp.tool()
def greet(name: str) -> str:
    """Generates a personalized greeting."""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run()  # stdio transport by default
'''
with open("/tmp/server.py", "w") as f:
    f.write(server_code)
print("Wrote /tmp/server.py")


### FastMCP over HTTP

In [0]:
import os, sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command=sys.executable,
    args=["/tmp/server.py"],
    env=os.environ.copy(),
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await session.list_tools()
        print("Tools:", [t.name for t in tools.tools])

        res = await session.call_tool("greet", {"name": "Databricks"})
        print("Result:", res.content)


In [0]:
server_code = r'''
from fastmcp import FastMCP

mcp_server = FastMCP()

@mcp_server.tool
def greet(name: str) -> str:
    """Generates a personalized greeting."""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp_server.run(transport="http", host="127.0.0.1", port=8000)
'''
with open("/tmp/server_http.py", "w") as f:
    f.write(server_code)
print("Wrote /tmp/server_http.py")

In [0]:
import subprocess, sys
proc = subprocess.Popen([sys.executable, "/tmp/server_http.py"])
print("server pid:", proc.pid)


In [0]:
proc.terminate()
proc.wait()

### Add exception handling + recovery

In [0]:
server_code = r'''
from mcp.server.fastmcp import FastMCP
import re

mcp = FastMCP("robust-location-server")

@mcp.tool()
def get_precise_location_info(address: str) -> dict:
    m = re.search(r"\b\d{5}\b", address)
    if not m:
        raise ValueError("No ZIP code found; precise lookup failed.")
    return {"ok": True, "type": "precise", "address": address, "zip": m.group(0)}

@mcp.tool()
def get_general_area_info(query: str) -> dict:
    parts = [p.strip() for p in query.split(",") if p.strip()]
    # "street, city, state" -> city=parts[1]
    # "city, state" -> city=parts[0]
    if len(parts) >= 2:
        city_guess = parts[1]
    elif len(parts) == 1:
        city_guess = parts[0]
    else:
        city_guess = query.strip()
    return {"ok": True, "type": "general", "city_guess": city_guess, "parts": parts}

if __name__ == "__main__":
    mcp.run()
'''
with open("/tmp/mcp_location_server.py", "w") as f:
    f.write(server_code)

print("Updated server.")

In [0]:
import os, sys, asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

import asyncio

def content_to_text(result) -> str:
    """
    Convert MCP content parts to a readable string.
    """
    parts = []
    for c in getattr(result, "content", []) or []:
        # TextContent has .text
        txt = getattr(c, "text", None)
        parts.append(txt if txt is not None else str(c))
    return "\n".join(parts).strip()

async def safe_call(session, tool_name: str, args: dict, timeout_s: float = 15.0):
    """
    Returns (ok: bool, payload: dict|str)
    ok=False if:
      - timeout/exception occurs, OR
      - MCP result has isError=True
    """
    try:
        res = await asyncio.wait_for(session.call_tool(tool_name, args), timeout=timeout_s)

        # ✅ MCP-level tool error (server returned error result)
        if getattr(res, "isError", False):
            return False, content_to_text(res) or f"Tool {tool_name} returned error."

        # ✅ success
        return True, res

    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

In [0]:
def extract_text_content(call_tool_result):
    """
    MCP results often return a list of 'content parts'.
    We'll stringify safely so you always see something usable.
    """
    try:
        return str(call_tool_result.content)
    except Exception:
        return str(call_tool_result)

In [0]:
async def run_robust_location_flow(user_query: str):
    state.clear()
    state["user_query"] = user_query
    state["primary_location_failed"] = False
    state["location_result"] = None

    server_params = StdioServerParameters(
        command=sys.executable,
        args=["/tmp/mcp_location_server.py"],
        env=os.environ.copy(),
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # (Optional) sanity: list tools once
            tools = await session.list_tools()
            tool_names = [t.name for t in tools.tools]
            # print("Tools:", tool_names)

            # ---- Primary attempt
            ok, res_or_err = await safe_call(session, "get_precise_location_info", {"address": user_query}, timeout_s=10.0)

            if ok:
                state["location_result"] = content_to_text(res_or_err)  # will be normal success content
                state["primary_location_failed"] = False
            else:
                state["primary_location_failed"] = True
                state["primary_error"] = res_or_err

                ok2, res2_or_err = await safe_call(session, "get_general_area_info", {"query": user_query}, timeout_s=10.0)
                if ok2:
                    state["location_result"] = content_to_text(res2_or_err)
                else:
                    state["fallback_error"] = res2_or_err


    # ---- Response phase (no tools; just state)
    if state.get("location_result"):
        if state["primary_location_failed"]:
            return (
                f"Primary lookup failed ({state.get('primary_error')}).\n"
                f"Fallback result:\n{state['location_result']}"
            )
        else:
            return f"Precise result:\n{state['location_result']}"
    else:
        return (
            "Sorry — I couldn't retrieve location information.\n"
            f"Primary error: {state.get('primary_error')}\n"
            f"Fallback error: {state.get('fallback_error')}"
        )

# Databricks notebooks already have an event loop → use top-level await
await run_robust_location_flow("1 Main St, New York, NY")


In [0]:
import json

def format_location_result(state: dict) -> str:
    result = state.get("location_result")
    if isinstance(result, str):
        return result
    return json.dumps(result, indent=2)

# in your final section:
if state.get("location_result"):
    if state["primary_location_failed"]:
        print(
            "Precise lookup couldn’t be completed (missing ZIP code).\n"
            f"Fallback (general) result:\n{format_location_result(state)}"
        )
    else:
        print(f"Precise result:\n{format_location_result(state)}")